In [6]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [7]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [8]:
from webdriver_manager.chrome import ChromeDriverManager
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--incognito")
DRIVER_PATH = ChromeDriverManager().install()
browser = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=chrome_options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\ACER.000\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
C:\Users\ACER.000\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [9]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [10]:
from webdriver_manager.utils import ChromeType


browser = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\ACER.000\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [11]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [13]:
import requests  
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver

In [14]:
def fetch_image_urls(query:str, max_links_to_fetch:500, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [15]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [16]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [17]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=116):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [18]:
import os
from PIL import Image
import io
import time
import hashlib
from time import sleep
search_term = 'extremely scared human face'
search_and_download(search_term=search_term, driver_path= DRIVER_PATH )

Found: 36 search results. Extracting links from 0:36
Found: 36 image links, looking for more ...


TypeError: 'NoneType' object is not iterable